#**4. Transaction Analysis for Market Expansion**

In [2]:
import sys
import os

sys.path.append(os.path.abspath("D:/D26_Files/Phonepe_Analytics/Pulse_Case_Studies/"))

from db_connection import get_phonepe_engine


engine = get_phonepe_engine()

In [3]:
import pandas as pd
from sqlalchemy import text

In [3]:
tables_df = pd.read_sql("SHOW TABLES;", engine)
tables_df

,Tables_in_phonepe_db
0,agg_transaction
1,agg_user
2,insurance_transaction
3,map_transaction
4,map_user
5,top_transaction


In [4]:
Agg_Transaction_df = pd.read_sql("SELECT * FROM agg_transaction;", engine)
Agg_Transaction_df.head()

,State,Year,Quarter,Transaction_type,Transaction_count,Transaction_amount
0,Andaman & Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman & Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman & Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05
3,Andaman & Nicobar Islands,2018,1,Financial Services,33,1.060142e+04
4,Andaman & Nicobar Islands,2018,1,Others,256,1.846899e+05


In [5]:
Map_Transaction_df = pd.read_sql("SELECT * FROM map_transaction;", engine)
Map_Transaction_df.head()

,State,Year,Quarter,District_name,Transaction_count,Transaction_amount
0,Andaman & Nicobar Islands,2018,1,North And Middle Andaman District,442,9.316631e+05
1,Andaman & Nicobar Islands,2018,1,South Andaman District,5688,1.256025e+07
2,Andaman & Nicobar Islands,2018,1,Nicobars District,528,1.139849e+06
3,Andaman & Nicobar Islands,2018,2,North And Middle Andaman District,825,1.317863e+06
4,Andaman & Nicobar Islands,2018,2,South Andaman District,9395,2.394824e+07


*1.	What is the overall contribution of states to PhonePe’s transaction volume & value?*

In [6]:
states_contribution_df = pd.read_sql("""
SELECT 
    State,
    SUM(Transaction_count) as Total_Transaction_Volume,
    SUM(Transaction_amount) as Total_Transaction_Value,
    ROUND(AVG(Transaction_amount/Transaction_count), 2) as Avg_Transaction_Value
FROM agg_transaction 
GROUP BY State 
ORDER BY Total_Transaction_Value DESC, Total_Transaction_Volume DESC;
""", engine)
states_contribution_df.head()

,State,Total_Transaction_Volume,Total_Transaction_Value,Avg_Transaction_Value
0,Telangana,2.617468e+10,4.165596e+13,1281.29
1,Karnataka,3.097095e+10,4.067872e+13,1215.76
2,Maharashtra,3.198521e+10,4.037420e+13,1155.92
3,Andhra Pradesh,1.891870e+10,3.466908e+13,1280.31
4,Uttar Pradesh,1.852360e+10,2.688521e+13,1164.27


*2.	How much do the top 5 states dominate compared to the rest of India?*

In [8]:
Top5_dominance_df = pd.read_sql("""
WITH state_totals AS (
    SELECT 
        State,
        SUM(Transaction_count) as Total_Volume,
        SUM(Transaction_amount) as Total_Value
    FROM agg_transaction 
    GROUP BY State
),
ranked_states AS (
    SELECT 
        State,
        Total_Volume,
        Total_Value,
        ROW_NUMBER() OVER (ORDER BY Total_Value DESC) as state_rank
    FROM state_totals
)
SELECT 
    CASE 
        WHEN state_rank <= 5 THEN 'Top 5 States'
        ELSE 'Rest of India'
    END as Category,
    SUM(Total_Volume) as Total_Transaction_Volume,
    SUM(Total_Value) as Total_Transaction_Value,
    COUNT(*) as Number_of_States,
    ROUND(SUM(Total_Value) * 100.0 / (SELECT SUM(Total_Value) FROM state_totals), 2) as Percentage_of_Total_Value,
    ROUND(SUM(Total_Volume) * 100.0 / (SELECT SUM(Total_Volume) FROM state_totals), 2) as Percentage_of_Total_Volume
FROM ranked_states
GROUP BY 
    CASE 
        WHEN state_rank <= 5 THEN 'Top 5 States'
        ELSE 'Rest of India'
    END
ORDER BY Total_Transaction_Value DESC;
""", engine)
Top5_dominance_df

,Category,Total_Transaction_Volume,Total_Transaction_Value,Number_of_States,Percentage_of_Total_Value,Percentage_of_Total_Volume
0,Top 5 States,1.265731e+11,1.842632e+14,5,53.33,53.8
1,Rest of India,1.087111e+11,1.612591e+14,31,46.67,46.2


*3.	Which underperforming states show strong recent growth (future opportunities)?*

In [11]:
underperforming_growth_states_df = pd.read_sql("""
SELECT 
    State,
    SUM(CASE WHEN Year = (SELECT MAX(Year) FROM agg_transaction) 
        THEN Transaction_amount ELSE 0 END) as Recent_Year_Value,
    SUM(CASE WHEN Year = (SELECT MAX(Year) - 1 FROM agg_transaction) 
        THEN Transaction_amount ELSE 0 END) as Previous_Year_Value,
    SUM(Transaction_amount) as Total_Overall_Value,
    ROUND(
        ((SUM(CASE WHEN Year = (SELECT MAX(Year) FROM agg_transaction) THEN Transaction_amount ELSE 0 END) - 
          SUM(CASE WHEN Year = (SELECT MAX(Year) - 1 FROM agg_transaction) THEN Transaction_amount ELSE 0 END)) * 100.0 /
         SUM(CASE WHEN Year = (SELECT MAX(Year) - 1 FROM agg_transaction) THEN Transaction_amount ELSE 0 END)), 2
    ) as Growth_Rate
FROM agg_transaction
GROUP BY State
HAVING Previous_Year_Value > 0
ORDER BY Growth_Rate DESC
LIMIT 10;
""", engine)
underperforming_growth_states_df

,State,Recent_Year_Value,Previous_Year_Value,Total_Overall_Value,Growth_Rate
0,Manipur,4.387420e+10,2.345805e+10,1.862656e+11,87.03
1,Lakshadweep,7.164300e+08,3.976678e+08,1.609321e+09,80.16
2,Jammu & Kashmir,5.284440e+11,3.115380e+11,1.184683e+12,69.62
3,Bihar,7.403791e+12,4.848213e+12,1.790135e+13,52.71
4,Ladakh,3.971840e+10,2.623368e+10,8.899462e+10,51.40
5,West Bengal,6.387113e+12,4.227142e+12,1.558416e+13,51.10
6,Arunachal Pradesh,1.224352e+11,8.145647e+10,2.744350e+11,50.31
7,Meghalaya,6.839180e+10,4.644904e+10,1.625439e+11,47.24
8,Andaman & Nicobar Islands,3.073972e+10,2.093302e+10,7.066745e+10,46.85
9,Chhattisgarh,2.033512e+12,1.388610e+12,4.890472e+12,46.44


*4.	Which states are “saturated” vs “emerging” markets for PhonePe?*

In [ ]:
market_status_df = pd.read_sql("""
SELECT 
    State,
    SUM(Transaction_amount) as Total_Value,
    ROUND(AVG(Transaction_amount/Transaction_count), 2) as Avg_Transaction_Size,
    ROUND(
        (MAX(CASE WHEN Year = (SELECT MAX(Year) FROM agg_transaction) THEN Transaction_amount END) -
         MAX(CASE WHEN Year = (SELECT MAX(Year) - 1 FROM agg_transaction) THEN Transaction_amount END))
        * 100.0 /
        MAX(CASE WHEN Year = (SELECT MAX(Year) - 1 FROM agg_transaction) THEN Transaction_amount END), 2) as Growth_Percentage
FROM agg_transaction
GROUP BY State
HAVING MAX(Year) >= (SELECT MAX(Year) - 1 FROM agg_transaction)
ORDER BY Total_Value DESC;
""", engine )
market_status_df

*5.	Which 10 states shows high Total transaction value vs Total transaction volume?*

In [ ]:
top_transaction_volume_df= pd.read_sql("""
 SELECT 
    State,
    SUM(Transaction_count) as Total_Transaction_Volume
FROM agg_transaction
GROUP BY State
ORDER BY Total_Transaction_Volume DESC
LIMIT 10;
""", engine)
top_transaction_volume_df

,State,Total_Transaction_Volume
0,Maharashtra,3.198521e+10
1,Karnataka,3.097095e+10
2,Telangana,2.617468e+10
3,Andhra Pradesh,1.891870e+10
4,Uttar Pradesh,1.852360e+10
5,Rajasthan,1.710854e+10
6,Madhya Pradesh,1.407218e+10
7,Bihar,1.094103e+10
8,West Bengal,9.191500e+09
9,Odisha,8.918527e+09


In [ ]:
top_transaction_value_df= pd.read_sql("""
 SELECT 
    State,
    SUM(Transaction_amount) as Total_Transaction_Value
FROM agg_transaction
GROUP BY State
ORDER BY Total_Transaction_Value DESC
LIMIT 10;
""", engine)
top_transaction_value_df

,State,Total_Transaction_Value
0,Telangana,4.165596e+13
1,Karnataka,4.067872e+13
2,Maharashtra,4.037420e+13
3,Andhra Pradesh,3.466908e+13
4,Uttar Pradesh,2.688521e+13
5,Rajasthan,2.634324e+13
6,Madhya Pradesh,1.912528e+13
7,Bihar,1.790135e+13
8,West Bengal,1.558416e+13
9,Odisha,1.226398e+13
